<a href="https://colab.research.google.com/github/luizabreu77/Repositorio_Imersao/blob/main/Intermod_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Lista de frequências de FM (canalização) de 87,5 MHz até 108 MHz com passo de 0,2 MHz
fm_frequencies = [round(87.5 + 0.2 * i, 1) for i in range(int((108.0 - 87.5) / 0.2) + 1)]

# Lista de frequências de canalização VHF (em MHz) de acordo com as bandas I e III
vhf_frequencies = [
    # Banda I: Canais 2 a 6
    (54, 60),  # Canal 2
    (60, 66),  # Canal 3
    (66, 72),  # Canal 4
    (76, 82),  # Canal 5
    (82, 88),  # Canal 6
    # Banda III: Canais 7 a 13
    (174, 180),  # Canal 7
    (180, 186),  # Canal 8
    (186, 192),  # Canal 9
    (192, 198),  # Canal 10
    (198, 204),  # Canal 11
    (204, 210),  # Canal 12
    (210, 216)   # Canal 13
]

# Frequências centrais dos canais UHF de 470 MHz a 806 MHz (espaçamento de 6 MHz)
uhf_frequencies = [round(470 + 6 * i + 3, 1) for i in range(int((806 - 470) / 6) + 1)]

# Função para gerar todas as frequências centrais dos canais VHF
def generate_vhf_center_freqs(vhf_channels):
    centers = []
    for start, end in vhf_channels:
        center = round((start + end) / 2, 1)
        centers.append(center)
    return centers

# Lista de frequências centrais dos canais VHF
vhf_center_frequencies = generate_vhf_center_freqs(vhf_frequencies)

# Função para encontrar intermodulações que geram o produto alvo com FM e VHF
def find_vhf_fm_intermodulations(fm_freqs, vhf_freqs, target, max_order):
    results = []
    for f1 in fm_freqs:
        for f2 in vhf_freqs:
            # Testa ordens de 3ª a 9ª
            for order in range(3, max_order + 1):
                m_values = range(1, order)  # m + n = ordem
                for m in m_values:
                    n = order - m
                    # Calcula as combinações possíveis de intermodulação
                    im1 = abs(m * f1 - n * f2)
                    im2 = abs(m * f2 - n * f1)
                    if round(im1, 1) == target or round(im2, 1) == target:
                        results.append({
                            'FM Frequency (MHz)': f1,
                            'VHF Frequency (MHz)': f2,
                            'Order': order,
                            'Intermodulation Product (MHz)': target
                        })
    return pd.DataFrame(results)

# Função para encontrar intermodulações que geram o produto alvo com FM, VHF e UHF
def find_uhf_vhf_fm_intermodulations(fm_freqs, vhf_freqs, uhf_freqs, target, max_order):
    results = []
    for f1 in fm_freqs:
        for f2 in vhf_freqs + uhf_freqs:  # Combinando VHF e UHF
            # Testa ordens de 3ª a 9ª
            for order in range(3, max_order + 1):
                m_values = range(1, order)  # m + n = ordem
                for m in m_values:
                    n = order - m
                    # Calcula as combinações possíveis de intermodulação
                    im1 = abs(m * f1 - n * f2)
                    im2 = abs(m * f2 - n * f1)
                    if round(im1, 1) == target or round(im2, 1) == target:
                        results.append({
                            'FM Frequency (MHz)': f1,
                            'TV Frequency (MHz)': f2,
                            'Order': order,
                            'Intermodulation Product (MHz)': target
                        })
    return pd.DataFrame(results)

# Encontrar produtos de intermodulação que resultam em 119.5 MHz com apenas FM e VHF
intermod_vhf_fm_results_119_5 = find_vhf_fm_intermodulations(fm_frequencies, vhf_center_frequencies, 119.5, 9)

# Encontrar produtos de intermodulação que resultam em 119.5 MHz com FM, VHF e UHF
intermod_uhf_vhf_fm_results_119_5 = find_uhf_vhf_fm_intermodulations(fm_frequencies, vhf_center_frequencies, uhf_frequencies, 119.5, 9)

# Combinar e remover duplicatas para apresentar o resultado final
combined_results = pd.concat([intermod_vhf_fm_results_119_5, intermod_uhf_vhf_fm_results_119_5]).drop_duplicates()

# Exibir o resultado final
print(combined_results)
